In [1]:
# Load the data
import pickle
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as netcdf

with open('ssp585_time_series.pkl', 'rb') as f:
    dic_ssp585 = pickle.load(f)

In [2]:
import os 

# Get the list of all files and directories
path = "/net/atmos/data/cmip6-ng/tos/ann/g025"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")

list_model = []
list_forcing = []

for idx, file in enumerate(dir_list):

    file_split = file.split("_")
    
    # extract model names
    model_name = file_split[2]
    forcing = file_split[3]
    run_name = file_split[4]
    
    list_model.append(model_name)
    list_forcing.append(forcing)
    
model_names = list(set(list_model))
forcing_names = list(set(list_forcing))

Files and directories in ' /net/atmos/data/cmip6-ng/tos/ann/g025 ' :


In [3]:
import netCDF4 as netcdf

# define the file
file = '/net/h2o/climphys3/simondi/cope-analysis/data/erss/sst_annual_g050_mean_19812014_centered.nc'

# read the dataset
file2read = netcdf.Dataset(file,'r')

# load longitude, latitude and sst monthly means
lon = np.array(file2read.variables['lon'][:])
lat = np.array(file2read.variables['lat'][:])
sst = np.array(file2read.variables['sst'])

# define grid
lat_grid, lon_grid = np.meshgrid(lat, lon, indexing='ij')

In [4]:
import skimage

# first filter out the models that do not contain ensemble members 
dic_reduced_ssp585 = {}

for m in list(dic_ssp585.keys()):
    if len(dic_ssp585[m].keys()) > 2:
        dic_reduced_ssp585[m] = dic_ssp585[m].copy()
        for idx_i, i in enumerate(dic_ssp585[m].keys()):
            dic_reduced_ssp585[m][i] = skimage.transform.downscale_local_mean(dic_reduced_ssp585[m][i],(1,2,2))

In [5]:
# second, for each model we compute the anomalies 
dic_processed_ssp585 = {}

import numpy as np

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    dic_processed_ssp585[m] = dic_reduced_ssp585[m].copy()
    
    mean_ref_ensemble = 0
    for idx_i, i in enumerate(dic_reduced_ssp585[m].keys()):
        
        if idx_i == 0:
            mean_ref_ensemble = np.nanmean(dic_processed_ssp585[m][i][131:164,:,:],axis=0)/ len(dic_processed_ssp585[m])
        else:
            mean_ref_ensemble += np.nanmean(dic_processed_ssp585[m][i][131:164,:,:],axis=0)/ len(dic_processed_ssp585[m])
    
    for idx_i, i in enumerate(dic_processed_ssp585[m].keys()):
        dic_processed_ssp585[m][i] = dic_processed_ssp585[m][i] - mean_ref_ensemble

/tmp/ipykernel_8238/4029070921.py:13: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble = np.nanmean(dic_processed_ssp585[m][i][131:164,:,:],axis=0)/ len(dic_processed_ssp585[m])
/tmp/ipykernel_8238/4029070921.py:15: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble += np.nanmean(dic_processed_ssp585[m][i][131:164,:,:],axis=0)/ len(dic_processed_ssp585[m])


In [6]:
# compute the forced response
dic_forced_response_ssp585 = dict({})

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    dic_forced_response_ssp585[m] = dic_reduced_ssp585[m].copy()
    
    mean_spatial_ensemble = 0
    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):
        
        if idx_i == 0:
            mean_spatial_ensemble = np.nanmean(dic_forced_response_ssp585[m][i],axis=(1, 2))/ len(dic_forced_response_ssp585[m])
        else:
            mean_spatial_ensemble += np.nanmean(dic_forced_response_ssp585[m][i],axis=(1, 2))/ len(dic_forced_response_ssp585[m])
            
    
    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):
        
        dic_forced_response_ssp585[m][i] = mean_spatial_ensemble - np.mean(mean_spatial_ensemble[131:164])

In [7]:
time_period = 33
grid_lat_size = 36
grid_lon_size = 72

y_forced_response = np.zeros((len(dic_forced_response_ssp585.keys()), time_period))
x_predictor = np.zeros((len(dic_forced_response_ssp585.keys()), time_period, grid_lat_size, grid_lat_size))

In [8]:
y_forced_response = {}
x_predictor = {}

for idx_m,m in enumerate(dic_processed_ssp585.keys()):
    y_forced_response[m] = 0
    x_predictor[m] = 0
    
    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):
        if idx_i ==0:
            y_forced_response[m] = dic_forced_response_ssp585[m][i][131:164]
            x_predictor[m] = dic_processed_ssp585[m][i][131:164,:,:]
        else:
            y_forced_response[m] = np.concatenate([y_forced_response[m],dic_forced_response_ssp585[m][i][131:164]])
            x_predictor[m] = np.concatenate([x_predictor[m], dic_processed_ssp585[m][i][131:164,:,:]],axis=0)        

In [9]:
# compute the variance
variance_processed_ssp585 = {}
std_processed_ssp585 = {}
for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    arr_tmp = np.zeros((len(dic_processed_ssp585[m].keys()),33))
    for idx_i, i in enumerate(dic_processed_ssp585[m].keys()):
        arr_tmp[idx_i,:] = np.nanmean(dic_processed_ssp585[m][i][131:164,:,:],axis=(1,2))
    variance_processed_ssp585[m] = np.mean(np.var(arr_tmp,axis=0))
    std_processed_ssp585[m] = np.mean(np.std(arr_tmp,axis=0))

In [10]:
import torch 

# Data preprocessing
x_train = {}
y_train = {}
a = 0
for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    x_train[m] = torch.from_numpy(np.nan_to_num(x_predictor[m]).reshape(x_predictor[m].shape[0],x_predictor[m].shape[1]*x_predictor[m].shape[2])).to(torch.float64)
    y_train[m] = torch.from_numpy(np.nan_to_num(y_forced_response[m])).to(torch.float64)

    nans_idx = np.where(np.isnan(x_predictor[m][0,:,:].ravel()))[0]

In [11]:
# Data processing 
for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    if idx_m == 0:
        X = x_train[m]
        y = y_train[m]
        D = torch.eye(x_train[m].shape[0])
    else:
        X = torch.cat((X,x_train[m]),0)
        y = torch.cat((y,y_train[m]),0)

In [14]:
def ridge_estimator(models,dic_x,dic_y,gammas,lambda_):
    """
    Compute the ridge estimator given gammas.
    """
    
    for idx_m,m in enumerate(models):
        if idx_m == 0:
            X = dic_x[m]
            y = dic_y[m]
            D = gammas[idx_m]*torch.eye(dic_x[m].shape[0])
        else:
            X = torch.cat((X,dic_x[m]),0)
            y = torch.cat((y,dic_y[m]),0)
            D_tmp = (gammas[idx_m] * torch.eye(dic_x[m].shape[0])).to(torch.float64)
            D = torch.block_diag(D, D_tmp).to(torch.float64)

    A = torch.matmul(torch.matmul(X.T, D),X) + lambda_ * torch.eye(X.shape[1])
    b = torch.matmul(torch.matmul(X.T,D),y)
    
    return torch.linalg.solve(A,b)

In [15]:
gammas = torch.rand(len(x_train.keys()))
lambda_ = 1.0
models = list(x_train.keys())

In [16]:
def residual_leave_one_out(m,dic_x,dic_y,vars,gammas,lambda_):

    # define list of models except m
    models_for_training = [model for model in list(dic_x.keys()) if model!=m]
    idx_models_for_training = [i for (i,m) in enumerate(list(dic_x.keys()))]

    # ridge estimator 
    beta = ridge_estimator(models_for_training,dic_x,dic_y,gammas[idx_models_for_training],lambda_)

    # compute residuals
    res = torch.mean((dic_y[m] - torch.matmul(dic_x[m],beta))**2/vars[m])

    return res

In [18]:
def train_leave_one_out_loss(x,y,vars,lon_size,lat_size,lambda_,nbEpochs,verbose=True):
    """
    Learn parameter β such that β = argmin( log Σ_m exp(||y_m - X_m^T β||^2) ).

    Args:
        - x: 
        - lon_size, lat_size: longitude and latitude grid size (Int)
        - models: (sub)list of models (list)
        - alpha_: softmax coefficient (float)
        - nbepochs: number of optimization steps (Int)
        - verbose: display logs (bool)
    """

    # define variable beta
    gammas = torch.zeros(len(x.keys())).to(torch.float64)
    gammas.requires_grad_(True)  
                          
    # define optimizer
    optimizer = torch.optim.Adam([gammas],lr=1e-3)
            
    # --- optimization loop ---                
    for epoch in torch.arange(nbEpochs):      
                      
        optimizer.zero_grad()
        ############### Define loss function ##############
                    
        # first term: ||Y - X - Rb ||
        obj = torch.tensor(0.0)
        for idx_m,m in enumerate(x.keys()):
            obj += residual_leave_one_out(m,x,y,vars,gammas,lambda_)
                
        #define loss function
        loss = obj
                    
        # Use autograd to compute the backward pass. 
        loss.backward(retain_graph=True)               
        
        # take a step into optimal direction of parameters minimizing loss
        optimizer.step()       
        
        if(verbose==True):
            if(epoch % 10 == 0):
                print('Epoch ', epoch.detach().item(), 
                    ', loss=', loss.detach().item()
                    )
    return gammas.detach().clone()

In [19]:
gammas = train_leave_one_out_loss(x_train,y_train,variance_processed_ssp585,\
                                grid_lon_size,grid_lat_size,\
                                lambda_,nbEpochs=1000,verbose=True)

0.5863480567932129
1.3211321830749512
tensor(10.9439, grad_fn=<AddBackward0>)
0.6774981021881104
1.4203529357910156
tensor(12.2878, grad_fn=<AddBackward0>)
0.6670641899108887
1.4541547298431396
tensor(24.8123, grad_fn=<AddBackward0>)
0.6538066864013672
1.4152662754058838
tensor(26.6054, grad_fn=<AddBackward0>)
0.626906156539917
1.3911826610565186
tensor(30.2619, grad_fn=<AddBackward0>)
0.604724645614624
1.3717055320739746
tensor(32.8551, grad_fn=<AddBackward0>)
0.6117734909057617
1.492027997970581
tensor(39.2694, grad_fn=<AddBackward0>)
0.7243959903717041
1.640211820602417
tensor(54.7499, grad_fn=<AddBackward0>)
0.6381833553314209
1.3816020488739014
tensor(56.5697, grad_fn=<AddBackward0>)
0.6299312114715576
1.4163093566894531
tensor(67.2886, grad_fn=<AddBackward0>)
0.5976588726043701
1.3479208946228027
tensor(67.7430, grad_fn=<AddBackward0>)
0.6080901622772217
1.4070143699645996
tensor(70.1748, grad_fn=<AddBackward0>)
0.6154577732086182
1.431072473526001
tensor(77.9356, grad_fn=<AddBac


KeyboardInterrupt

